In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense 
from tensorflow.keras.layers import GRU, Dense

2024-04-01 12:59:29.364935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 12:59:29.562674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 12:59:29.564187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 12:59:30.483979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
df = pd.read_csv('sredjeni_kurs.csv', parse_dates=True)
print(df.head())

  APPLICATION DATE  MIDDLE EXCHANGE RATE
0       2003-01-01               61.5845
1       2003-01-02               61.5845
2       2003-01-03               61.5845
3       2003-01-04               61.6615
4       2003-01-05               61.6615


In [5]:
# Pretvaranje podataka u vremensku seriju
time_series = df['MIDDLE EXCHANGE RATE'].values.astype(float)

In [6]:
# Normalizacija podataka
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series.reshape(-1, 1))

In [7]:
time_series_normalized

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.89038312],
       [0.89038312],
       [0.89038312]])

In [8]:
# Funkcija za pretvaranje vremenske serije u uzorke i ciljeve
def create_dataset(time_series, time_steps):
    X, y = [], []
    for i in range(len(time_series) - time_steps):
        X.append(time_series[i:(i + time_steps)])
        y.append(time_series[i + time_steps])
    return np.array(X), np.array(y)

In [10]:
# Postavljanje parametara
time_steps = 30  
batch_size = 64
epochs = 100

# Kreiranje uzoraka i ciljeva
X, y = create_dataset(time_series_normalized, time_steps)

In [11]:
# Podela podataka na skupove za obuku i testiranje
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Kreiranje modela
model = Sequential([
    GRU(50, input_shape=(time_steps, 1)),
    Dense(1)
])

# Kompilacija modela
model.compile(optimizer='adam', loss='mean_squared_error')

# Treniranje modela
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/100
96/96 [==============================] - 3s 11ms/step - loss: 0.0422
Epoch 2/100
96/96 [==============================] - 1s 11ms/step - loss: 5.8210e-04
Epoch 3/100
96/96 [==============================] - 1s 11ms/step - loss: 4.0755e-05
Epoch 4/100
96/96 [==============================] - 1s 11ms/step - loss: 3.8296e-05
Epoch 5/100
96/96 [==============================] - 1s 11ms/step - loss: 3.6076e-05
Epoch 6/100
96/96 [==============================] - 1s 11ms/step - loss: 3.4411e-05
Epoch 7/100
96/96 [==============================] - 1s 11ms/step - loss: 3.2276e-05
Epoch 8/100
96/96 [==============================] - 1s 11ms/step - loss: 3.0634e-05
Epoch 9/100
96/96 [==============================] - 1s 11ms/step - loss: 2.9269e-05
Epoch 10/100
96/96 [==============================] - 1s 11ms/step - loss: 2.8270e-05
Epoch 11/100
96/96 [==============================] - 1s 11ms/step - loss: 2.7012e-05
Epoch 12/100
96/96 [==============================] - 1s 11ms/step 

96/96 [==============================] - 1s 11ms/step - loss: 1.8770e-05
Epoch 97/100
96/96 [==============================] - 1s 12ms/step - loss: 1.6059e-05
Epoch 98/100
96/96 [==============================] - 1s 11ms/step - loss: 1.6103e-05
Epoch 99/100
96/96 [==============================] - 1s 11ms/step - loss: 1.8171e-05
Epoch 100/100
96/96 [==============================] - 1s 11ms/step - loss: 1.7261e-05


In [13]:
# Evaluacija modela
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)

48/48 [==============================] - 0s 4ms/step - loss: 1.4348e-05
Test Loss: 1.4347906471812166e-05


In [14]:
# Poslednjih 30 vremenskih koraka iz testnog skupa
X_future = X_test[-30:]

# Predviđanje kretanja kursa dinara za budući period
future_predictions_normalized = model.predict(X_future)
future_predictions = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions:', future_predictions)


1/1 [==============================] - 0s 229ms/step
Future Predictions: [[117.5808  ]
 [ 65.22695 ]
 [106.06819 ]
 [ 87.588104]
 [117.550995]
 [118.054825]
 [113.65062 ]
 [117.17541 ]
 [ 71.484276]
 [122.86855 ]
 [ 96.74774 ]
 [ 93.12359 ]
 [113.79178 ]
 [120.6537  ]
 [120.50632 ]
 [117.42316 ]
 [ 77.69711 ]
 [123.27148 ]
 [ 81.66505 ]
 [117.58306 ]
 [118.377266]
 [117.967354]
 [ 78.70222 ]
 [117.56823 ]
 [117.93369 ]
 [ 86.7988  ]
 [ 94.76528 ]
 [117.40353 ]
 [ 80.82337 ]
 [ 65.360855]]


In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Izračunaj predviđanja za testni skup podataka
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)

mse = mean_squared_error(y_test, predictions)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)

48/48 [==============================] - 0s 4ms/step
Mean Absolute Error (MAE): 0.0017131773764339794
Mean Squared Error (MSE): 1.4347903001924308e-05


# RNN

In [17]:
from tensorflow.keras.layers import LSTM, Dense

In [18]:
# Kreiranje modela
model_rnn = Sequential([
    LSTM(50, input_shape=(time_steps, 1)),
    Dense(1)
])

# Kompilacija modela
model_rnn.compile(optimizer='adam', loss='mean_squared_error')

# Treniranje modela
model_rnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)


Epoch 1/100
96/96 [==============================] - 2s 12ms/step - loss: 0.0252
Epoch 2/100
96/96 [==============================] - 1s 12ms/step - loss: 1.0578e-04
Epoch 3/100
96/96 [==============================] - 1s 12ms/step - loss: 7.8094e-05
Epoch 4/100
96/96 [==============================] - 1s 12ms/step - loss: 7.7942e-05
Epoch 5/100
96/96 [==============================] - 1s 11ms/step - loss: 7.7281e-05
Epoch 6/100
96/96 [==============================] - 1s 12ms/step - loss: 7.6239e-05
Epoch 7/100
96/96 [==============================] - 1s 11ms/step - loss: 7.5972e-05
Epoch 8/100
96/96 [==============================] - 1s 12ms/step - loss: 7.4859e-05
Epoch 9/100
96/96 [==============================] - 1s 12ms/step - loss: 7.4316e-05
Epoch 10/100
96/96 [==============================] - 1s 12ms/step - loss: 7.3687e-05
Epoch 11/100
96/96 [==============================] - 1s 12ms/step - loss: 7.3604e-05
Epoch 12/100
96/96 [==============================] - 1s 12ms/step 

96/96 [==============================] - 1s 12ms/step - loss: 3.0081e-05
Epoch 97/100
96/96 [==============================] - 1s 12ms/step - loss: 2.7862e-05
Epoch 98/100
96/96 [==============================] - 1s 12ms/step - loss: 2.6350e-05
Epoch 99/100
96/96 [==============================] - 1s 12ms/step - loss: 3.1052e-05
Epoch 100/100
96/96 [==============================] - 1s 12ms/step - loss: 2.8878e-05


In [19]:
# Evaluacija modela
loss = model_rnn.evaluate(X_test, y_test)
print('Test Loss:', loss)

48/48 [==============================] - 1s 5ms/step - loss: 2.3321e-05
Test Loss: 2.3320581021835096e-05


In [20]:
# Poslednjih 30 vremenskih koraka iz testnog skupa
X_future = X_test[-30:]

# Predviđanje kretanja kursa evra za budući period
future_predictions_normalized_rnn = model_rnn.predict(X_future)
future_predictions_rnn = scaler.inverse_transform(future_predictions_normalized)
print('Future Predictions:', future_predictions_rnn)


1/1 [==============================] - 0s 270ms/step
Future Predictions: [[117.5808  ]
 [ 65.22695 ]
 [106.06819 ]
 [ 87.588104]
 [117.550995]
 [118.054825]
 [113.65062 ]
 [117.17541 ]
 [ 71.484276]
 [122.86855 ]
 [ 96.74774 ]
 [ 93.12359 ]
 [113.79178 ]
 [120.6537  ]
 [120.50632 ]
 [117.42316 ]
 [ 77.69711 ]
 [123.27148 ]
 [ 81.66505 ]
 [117.58306 ]
 [118.377266]
 [117.967354]
 [ 78.70222 ]
 [117.56823 ]
 [117.93369 ]
 [ 86.7988  ]
 [ 94.76528 ]
 [117.40353 ]
 [ 80.82337 ]
 [ 65.360855]]


In [21]:
# Izračunaj predviđanja za testni skup podataka
predictions_rnn = model_rnn.predict(X_test)

mae = mean_absolute_error(y_test, predictions_rnn)

mse = mean_squared_error(y_test, predictions_rnn)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)

48/48 [==============================] - 0s 5ms/step
Mean Absolute Error (MAE): 0.0027783100755762627
Mean Squared Error (MSE): 2.332058208998124e-05
